# Reseaux de neurones simples a une couches

In [3]:
import numpy as np

In [4]:
def initialisation(n0,n1,n2):
    W1 = np.random.randn(n1,n0)
    b1 = np.random.randn(n1,1)

    W2 = np.random.randn(n2,n1)
    b2 = np.random.randn(n2,1)

    parameteres={
        'W1':W1,
        'b1':b1,
        'W2':W2,
        'b2':b2
    }
    return parameteres

In [5]:
def froward_propagation(parametres,X):

    W1 = parametres['W1']
    b1 = parametres['b1']

    W2 = parametres['W2']
    b2 = parametres['b2']

    Z1 = W1.dot(X)+b1
    A1 = 1/(1+np.exp(-Z1))

    Z2 = W2.dot(A1)+b2
    A2 = 1/(1+np.exp(-Z2))

    activations={
        'A1' : A1,
        'A2' : A2
    }

    return activations

In [6]:
def backward_propagation(parametres,X,activations,y):
    A1 = activations['A1']
    A2 = activations['A2']

    W1 = parametres['W1']
    b1 = parametres['b1']

    W2 = parametres['W2']
    b2 = parametres['b2']

    m = y.shape[1]

    dZ2 = (A2-y)
    dW2 = (1/m)*dZ2.dot(A1.T)
    db2 = (1/m)*np.sum(dZ2, axis = 1,keepdims =True)
    dZ1 = np.dot(W2.T,dZ2)*A1*(1-A1)
    dW1 = (1/m)*dZ1.dot(X.T)
    db1 = (1/m)*np.sum(dZ1,axis = 1,keepdims =True)

    gradients = {
        "dW1" : dW1,
        "dW2" : dW2,
        "db1" : db1,
        "db2" : db2
    }

    return gradients
    





In [7]:
def update(gradients,parametres,learning_rate):
    
    W1 = parametres['W1']
    dW1 = gradients['dW1']
    b1 = parametres['b1']
    db1 = gradients['db1']

    W2 = parametres['W2']
    dW2 = gradients['dW2']
    b2 = parametres['b2']
    db2 = gradients['db2']

    W1 = W1 - learning_rate*dW1
    W2 = W2 - learning_rate*dW2

    b1 = b1 - learning_rate*db1
    b2 = b2 - learning_rate*db2

    parameteres={
        'W1':W1,
        'b1':b1,
        'W2':W2,
        'b2':b2
    }
    return parameteres